*Los ejercicios se basan en el conjunto de viviendas "housing.csv"*

### Ejercicio 3

Intente agregar un transformador **SelectFromModel** en el pipeline de preparación para seleccionar solo los atributos más importantes.

#### Paso 1: Preparando los datos

In [31]:
import requests
import tarfile

URL = "https://mymldatasets.s3.eu-de.cloud-object-storage.appdomain.cloud/housing.tgz"
PATH = "housing.tgz"

def getData(url=URL, path=PATH):
  r = requests.get(url)
  with open(path, 'wb') as f:
    f.write(r.content)
  housing_tgz = tarfile.open(path)
  housing_tgz.extractall()
  housing_tgz.close()

getData()

In [32]:
import pandas as pd

data = pd.read_csv('housing.csv')
data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [33]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [34]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2) # 20% para test y 80% para entrenamiento

len(data), len(train), len(test)

(20640, 16512, 4128)

#### Paso 2: Tratar los valores inexistentes ("missing values")

In [35]:
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)

In [36]:
train_data, y_train = train.drop(['median_house_value'], axis=1), train['median_house_value'].copy() # quitar columna de targets del dataset de entrenamiento
test_data, y_test = test.drop(['median_house_value'], axis=1), test['median_house_value'].copy() # agregar la columna de targets al set

# separar variables en numéricas y categóricas
train_num = train_data.drop(['ocean_proximity'], axis=1)
train_cat = train_data[['ocean_proximity']]

In [37]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median") # definir imputer
imputer.fit(train_num) # calcular mediana

# mediana de cada característica o columna (7)
imputer.statistics_ # valores calculados

array([-118.485 ,   34.25  ,   29.    , 2140.    ,  436.    , 1168.5   ,
        411.    ,    3.5417])

In [38]:
# llamamos a transform y le pasamos los valores que debe sustituir en cada característica

X_train_num = imputer.transform(train_num) # cambiar valores inexistentes por la media

X_train_num

array([[-1.2246e+02,  3.8530e+01,  3.2000e+01, ...,  7.8500e+02,
         3.0900e+02,  3.6641e+00],
       [-1.2124e+02,  3.8750e+01,  5.0000e+00, ...,  3.6670e+03,
         1.2940e+03,  5.4896e+00],
       [-1.2223e+02,  3.7760e+01,  5.2000e+01, ...,  8.0500e+02,
         3.2100e+02,  4.7188e+00],
       ...,
       [-1.1834e+02,  3.4190e+01,  4.3000e+01, ...,  6.1300e+02,
         2.5500e+02,  2.6827e+00],
       [-1.2193e+02,  3.8310e+01,  2.5000e+01, ...,  8.5000e+01,
         3.2000e+01,  4.8750e+00],
       [-1.2243e+02,  3.7760e+01,  5.2000e+01, ...,  8.6100e+02,
         4.0600e+02,  4.4318e+00]])

#### Paso 3: Preprocesado

In [39]:
## PREPROCESADO DE LAS VARIABLES NUMÉRICAS (escalado)

from sklearn.preprocessing import StandardScaler # también hay min-max

scaler = StandardScaler() # mean y std
scaler.fit(X_train_num)
X_train_num_scaled = scaler.transform(X_train_num)
X_train_num_scaled

array([[-1.4479531 ,  1.36237038,  0.26593831, ..., -0.57477376,
        -0.49988848, -0.10907815],
       [-0.8391341 ,  1.4653877 , -1.87698795, ...,  1.99190044,
         2.05386967,  0.84955467],
       [-1.33317574,  1.00180979,  1.8532911 , ..., -0.55696201,
        -0.46877671,  0.44478109],
       ...,
       [ 0.6080586 , -0.66988024,  1.13898234, ..., -0.72795487,
        -0.63989147, -0.62444504],
       [-1.18346615,  1.25935307, -0.28963517, ..., -1.19818526,
        -1.21805194,  0.52680708],
       [-1.43298214,  1.00180979,  1.8532911 , ..., -0.50708909,
        -0.24840164,  0.29406752]])

In [40]:
## PREPROCESADO DE LAS VARIABLES CATEGÓRICAS (OneHotEncoding)

from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder()
X_train_cat = cat_encoder.fit_transform(train_cat)
X_train_cat

<16512x5 sparse matrix of type '<class 'numpy.float64'>'
	with 16512 stored elements in Compressed Sparse Row format>

In [41]:
X_train_cat.toarray()

array([[0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.]])

#### Paso 4: Pipelines

In [42]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

## >> CREACIÓN
# inicializamos el pipeline y le pasamos una lista con los pasos a ejecutar

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")), # 1- imputer (rellenar huecos)
        ('std_scaler', StandardScaler()), # 2- función para normalizar (escalado)
    ])

##  >> SEPARACIÓN 
# separo los atributos en numéricos y categóricos

num_attribs = list(train_num) # atributos numéricos
cat_attribs = ["ocean_proximity"] # atributos categóricos

##  >> APLICACIÓN
# tenemos la posibilidad de aplicar diferentes pipelines a diferentes columnas

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs), # aplico "num_pipeline" a las columnas numéricas
    ("cat", OneHotEncoder(), cat_attribs), # aplico OneHotEncoder a las columnas categóricas
])

In [43]:
X_train = full_pipeline.fit_transform(train_data)

X_train # contiene las variables numéricas y categóricas, sin missing values y todo escalado y codificado.

array([[-1.4479531 ,  1.36237038,  0.26593831, ...,  0.        ,
         0.        ,  0.        ],
       [-0.8391341 ,  1.4653877 , -1.87698795, ...,  0.        ,
         0.        ,  0.        ],
       [-1.33317574,  1.00180979,  1.8532911 , ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [ 0.6080586 , -0.66988024,  1.13898234, ...,  0.        ,
         0.        ,  0.        ],
       [-1.18346615,  1.25935307, -0.28963517, ...,  0.        ,
         0.        ,  0.        ],
       [-1.43298214,  1.00180979,  1.8532911 , ...,  0.        ,
         1.        ,  0.        ]])

In [44]:
X_test = full_pipeline.transform(test_data) # ojo ! aquí no hacemos fit :) sólo transform

X_test

array([[ 0.59807796, -0.74011932,  1.53582054, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.67293276, -0.85718444,  1.21834998, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.31362974, -0.59027595,  0.34530595, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.78271979, -0.78226276,  0.10720303, ...,  0.        ,
         0.        ,  0.        ],
       [-1.12358232,  0.78172735, -1.08331156, ...,  0.        ,
         0.        ,  0.        ],
       [-1.20841775,  0.78172735,  0.18657067, ...,  0.        ,
         0.        ,  0.        ]])

#### Paso 5: Entrenar el modelo (aquí viene el cambio)

Usamos **GridSearchCV** para que nos busque automáticamente el **mejor modelo y el mejor hiperperámetro**,  **combinando todos los hiperparámetros**, uno a uno, sin tener que ajustarlos manualmente.

+ **Ventajas**: Mucho tiempo para probar todas las combinaciones.
- **Desventajas**: Se obtienen los mejores hiperparámetros.

Usamos **RandomSearchCV** para obtener una búsqueda aleatoria de un determinado número de combinaciones (iteraciones) dentro de un rango de valores.

+ **Ventajas**: Más rápido.
- **Desventajas**: No garantiza la mejor combinación porque no todos los valores se prueban.

Usamos **Cross-Validation** a la hora de entranar, lo que permite, para el set de entrenamiento, dejar siempre un subconjunto de datos (test) perteneciente el conjunto de entrenamiento en cada iteración e ir rotándolo cada vez.

Se usa para sacarle más provecho al set de datos cuando este es pequeño.

In [46]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import expon, loguniform

# Nota: gamma es ignorado cuando el kernel es "linear"

param_distribs = {
        'svr__kernel': ['linear', 'rbf'],
        'svr__C': loguniform(20, 200_000),
        'svr__gamma': expon(scale=1.0),
    }

rnd_search = RandomizedSearchCV(svr_pipeline,
                                param_distributions=param_distribs,
                                n_iter=50, cv=3,
                                scoring='neg_root_mean_squared_error',
                                random_state=42,
                                n_jobs=4)

rnd_search.fit(X_train[:5000], y_train[:5000])

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('num_pipeline',
                                              Pipeline(steps=[('imputer',
                                                               SimpleImputer(strategy='median')),
                                                              ('std_scaler',
                                                               StandardScaler())])),
                                             ('svr', SVR())]),
                   n_iter=50, n_jobs=4,
                   param_distributions={'svr__C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7fa7f56f0460>,
                                        'svr__gamma': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7fa7f567f7c0>,
                                        'svr__kernel': ['linear', 'rbf']},
                   random_state=42, scoring='neg_root_mean_squared_error')

In [47]:
# Impresión del mejor estimador

#print(mejor_modelo.best_estimator_.get_params()["svr__kernel"])
print(mejor_modelo.best_estimator_)

# El que mejor se adapta es SVR con kernel linear, C=3000.

Pipeline(steps=[('num_pipeline',
                 Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                                 ('std_scaler', StandardScaler())])),
                ('svr', SVR(C=10000.0, kernel='linear'))])


#### Paso 6: Obtener predicciones

In [49]:
# Predecir valores para todos los datos del test

y_test_pred = rnd_search.predict(X_test)
#y_test_prob = grid_search.predict_proba(X_test)

y_test_pred

array([174394.25867546, 124343.08071145, 215126.35697703, ...,
       488602.01055101, 268757.54500317, 263097.29949441])

In [ ]:
# OBTENER LAS MÉTRICAS

#auc = roc_auc_score(y_test, y_test_prob[:,1])
#print("- Precision: ", round(precision_score(y_test, y_test_pred), 2))
#print("- Recall: ", round(recall_score(y_test, y_test_pred), 2))
#print("- F-Score: ", round(f1_score(y_test, y_test_pred), 2))
#print("- AUC: ", round(auc, 2)) #área bajo la curva

In [50]:
# Obtener la puntuación del mejor modelo.

puntuacion_mejor_modelo = -rnd_search.best_score_

In [51]:
puntuacion_mejor_modelo

60059.718721644276

Ahora es mucho mejor pero aún está lejos del rendimiento de **RandomForestRegressor**.

Los mejores hiperparámetros son:

In [53]:
# Obtener los < MEJORES HIPERPARÁMETROS >

rnd_search.best_params_

{'svr__C': 157055.10989448498,
 'svr__gamma': 0.26497040005002437,
 'svr__kernel': 'rbf'}

El mejor conjuntos de hiperparámetros para el kernel es **RBF**.

La búsqueda aleatoria (**RandomSearchCV**) tiende a encontrar mejores hiperparámetros que la búsqueda rejilla (**GridSearchCV**) en la misma cantidad de tiempo.


--------------------------------------------------------
Usamos la distribución `expon()` para **gamma**, con una escala de 1, por lo que RandomSearch buscó principalmente valores de esa escala:

Alrededor del 80% de las muestras estaban entre 0,1 y 2,3 (aproximadamente el 10% eran más pequeñas y el 10% eran más grandes).

--------------------------------------------------------

In [55]:
import numpy as np

np.random.seed(42)

s = expon(scale=1).rvs(100_000)  # get 100,000 samples
((s > 0.105) & (s < 2.29)).sum() / 100_000

0.80066

Usamos las distribución `longuniform()` para C, lo que significa que no teníamos ni idea de cuál era la escala óptima de C antes de ejecutar la búsqueda aletoria.

Exploró el rango de 20 a 200 tanto como el rango de 2000 a 20.000 o de 20.000 a 200.000